# Start database

Set password and port

In [1]:
import getpass

In [ ]:
password = getpass.getpass()
port=7777

Run PostgreSQL container with shell script block

The `%%srcipt bash` magic command execute `bash` code in the cell. It can be used with all popular scripting languages, such as `R`, `perl` and many more.

- `--bg`: The script will run in the background, without it the executed cell would not give back the control to you until killed. 

- `-s $password $port` pass the `password` and `port` python variable to the bash script. They can be refered a `"$1"` and `"$2"` respectivly.

In [ ]:
%%script bash --bg -s $password $port
set -e -u -o pipefail

password="$1"
port="$2"

test -d data || mkdir data 

docker run -d \
    --name playground-postgres \
    --rm \
    --shm-size=512m \
    -e POSTGRES_PASSWORD="$password" \
    -e PGDATA=/var/lib/postgresql/data/pgdata \
    -v data:/var/lib/postgresql/data \
    -p "$port":5432 \
    postgres

`set -e -u -o pipefail` bash strict mode makes the script halt:
 - on any error `-e`, 
 - even in a pipeline `-o pipefail`
 - undefined variable `-u` (otherwise would be empty a string)
 
Can be written as `set -euo pipefail` for short. Without these options shell scripts can be very dangerous.

This script has no pipelines (`command1 | command2`) and has only one command. So the `-e` and `-o piepfail` options cannot have any effect, still it is a best practice too always use them, because the script will maybe modified in the future.

In [ ]:
%%script bash

docker container ls | grep playground

Test PostgreSQL access by listing all logical databases

In [ ]:
%%script bash
set -e -u -o pipefail

docker exec \
    -i \
    playground-postgres \
    psql -U postgres << EOF
\l
EOF

# Load the Pagila sample database

![](./pagila_schema.png)

In [ ]:
%%script bash
set -e -u -o pipefail

docker exec \
    -i \
    playground-postgres \
    bash << EOF
set -e -u -o pipefail
apt-get update &> /dev/null
apt-get install -y curl &> /dev/null
createdb -U postgres pagila
curl https://raw.githubusercontent.com/morenoh149/postgresDBSamples/master/pagila-0.10.1/pagila-schema.sql | psql -d pagila -U postgres
curl https://raw.githubusercontent.com/morenoh149/postgresDBSamples/master/pagila-0.10.1/pagila-data.sql | psql -d pagila -U postgres
EOF

# Access to the database with SQLAlchemy

In [ ]:
user = "postgres"
server = "localhost"
database = "pagila"
driver = "postgres_utf8"
connection_string = f"postgresql://{user}:{password}@{server}:{port}/{database}"

In [ ]:
import sqlalchemy as sa

In [ ]:
engine = sa.create_engine(connection_string)
engine

In [ ]:
c = engine.connect()

In [ ]:
import pandas as pd

In [ ]:
df_nut_data = pd.read_sql(
    """
    SELECT
        *
    FROM
        actor
    LIMIT 10
    """,
    c
)
df_nut_data

# Access to the database with ipython-sql kernel

ipython-sql kernal uses SQLAlchemy in the background so its need the sam connection string.

In [ ]:
%load_ext sql
%sql $connection_string
%config SqlMagic.autopandas=True

List all table in the database

In [ ]:
%sql \dt

In [ ]:
Read `data_src` table

In [ ]:
%%sql

SELECT
    *
FROM
    data_src
LIMIT 10

In [ ]:
Read `data_src` table into a dataframe

In [ ]:
%%sql df <<

SELECT
    *
FROM
    data_src

In [ ]:
df

# Stop database

By killing all background scripts

In [ ]:
%%script bash

docker container kill playground-postgres

In [ ]:
%killbgscripts